In [6]:
import pandas as pd
import numpy as np

from column_report import get_column_report
###### Now adding census data to dataframe!! 

# data from:
#     Steven Manson, Jonathan Schroeder, David Van Riper, and Steven Ruggles. 
#     IPUMS National Historical Geographic Information System: 
#     Version 12.0 [Database]. Minneapolis: University of Minnesota. 2017. http://doi.org/10.18128/D050.V12.0

nhgis = pd.read_csv('data/nhgis0003_csv/nhgis0003_ds172_2010_block.csv', low_memory=False)
                    #usecols=[12,28,32,34,35,36,38,39,40,48,51,54,55,56,57,58,62,63,65])



%store nhgis
#%store -r nhgis


Stored 'nhgis' (DataFrame)


In [2]:
# create a dictionary of the original full list of columns and their descriptions
full_col_dict = dict(nhgis.iloc[0])

# for col in full_col_dict:
#     print(col, ':', full_col_dict[col])

In [21]:
ahs_df = nhgis[nhgis.columns[[10,6,11,12,28,32,40,48,51,54,55,56,62]]]
# columns and their descriptions that we want to use
for i, col in enumerate(ahs_df.columns):
    print(i,'%s: ' %col, ahs_df[col][0])

# remove descriptive header row
#ahs_df = ahs_df[1:]
# and rename columns
ahs_df.columns = ['tract','county','blkgroup', 'census_block', 'rur_urb','zip','population','median_age','avg_household_size',
                 'num_housing_units','occupied','vacant','seasonal']


# # we're only interested in the san francisco data
ahs_df = ahs_df[ahs_df.county == 'San Francisco County']

# # each row is identified by (tract, block)
print(len(ahs_df.groupby(['census_block', 'tract']).size()))


0 TRACTA:  Census Tract Code
1 COUNTY:  County Name
2 BLKGRPA:  Block Group Code
3 BLOCKA:  Block Code
4 URBRURALA:  Urban/Rural Code
5 ZCTA5A:  5-Digit Zip Code Tabulation Area Code
6 H7X001:  Total
7 H77001:  Median age: Both sexes
8 H8B001:  Average household size: Total
9 IFE001:  Total
10 IFE002:  Occupied
11 IFE003:  Vacant
12 IFG006:  For seasonal, recreational, or occasional use
7386


In [22]:
# in order to match with masterdf, need to change dtypes to numeric
for c in ahs_df.columns:
    if ((c!='rur_urb')&(c!='county')):
        ahs_df[c] = pd.to_numeric(ahs_df[c])

# NaNs here are just zeroes, so doesn't always imply missing data
# but it does in population, median_age, 
get_column_report(ahs_df)

,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
0,tract,int64,197,10100,0,0.00
1,county,object,1,San Francisco County,0,0.00
3,census_block,int64,404,1000,0,0.00
4,rur_urb,object,2,R,0,0.00
5,zip,int64,28,94133,0,0.00
2,blkgroup,int64,8,1,6,0.08
6,population,int64,539,0,2249,30.45
7,median_age,float64,442,0,2249,30.45
9,num_housing_units,int64,364,0,2450,33.17
8,avg_household_size,float64,403,0,2469,33.43


In [23]:
# grabbing the extra features I missed the first time around

import glob

path = 'data/nhgis0004_csv/'

files = glob.glob(path + '*.csv')

dfs = []

for file in files:
    df = pd.read_csv(file, encoding='ISO-8859-1')
    dfs.append(df)
    
[print(f) for f in files]

# and this tells us the number of people under 18 in each block GROUP in 2010
under18 = dfs[-2][['COUNTY', 'BLCK_GRPA', 'TRACTA', 'CR0AA2010']].rename(columns={'CR0AA2010':'num_under_18'})

under18 = under18[under18.COUNTY == 'San Francisco County']

#print(under18.columns)

for c in under18.columns:
    if (c!='COUNTY'):
        under18[c] = pd.to_numeric(under18[c])




under18.head()


data/nhgis0004_csv/nhgis0004_ds176_20105_2010_blck_grp.csv
data/nhgis0004_csv/nhgis0004_ds176_20105_2010_tract.csv
data/nhgis0004_csv/nhgis0004_ds215_20155_2015_blck_grp.csv
data/nhgis0004_csv/nhgis0004_ds215_20155_2015_tract.csv
data/nhgis0004_csv/nhgis0004_ds216_20155_2015_tract.csv
data/nhgis0004_csv/nhgis0004_ts_geog2010_blck_grp.csv
data/nhgis0004_csv/nhgis0004_ts_geog2010_tract.csv


,COUNTY,BLCK_GRPA,TRACTA,num_under_18
28133,San Francisco County,1,10100,8
28134,San Francisco County,2,10100,421
28135,San Francisco County,1,10200,115
28136,San Francisco County,2,10200,84
28137,San Francisco County,3,10200,56


In [24]:
# now merge that into the main census df

ahs_df = ahs_df.merge(under18[['num_under_18','BLCK_GRPA', 'TRACTA']], left_on=['blkgroup', 'tract'],
                      right_on=['BLCK_GRPA','TRACTA'], how='left')



In [25]:
# same thing for public assistance income

pub_assist_income = dfs[-5][dfs[-5].COUNTY=='San Francisco County'][['BLKGRPA','TRACTA', 'ADN4E001','ADN4E002','ADN4E003']] \
                                                                    .rename(columns={'ADN4E001':'total_hholds', \
                                                                         'ADN4E002':'hholds_pubassist', \
                                                                         'ADN4E003':'hholds_no_pubassist'})

    
for c in pub_assist_income.columns:
    if (c!='COUNTY'):
        pub_assist_income[c] = pd.to_numeric(pub_assist_income[c])



ahs_df = ahs_df.merge(pub_assist_income, left_on=['blkgroup','tract'],
                      right_on=['BLKGRPA', 'TRACTA'],how='left') \
                    .drop(['BLCK_GRPA','BLKGRPA', 'TRACTA_x', 'TRACTA_y'],axis=1)

ahs_df.head()

,tract,county,blkgroup,census_block,rur_urb,zip,population,median_age,avg_household_size,num_housing_units,occupied,vacant,seasonal,num_under_18,total_hholds,hholds_pubassist,hholds_no_pubassist
0,10100,San Francisco County,1,1000,R,94133,0,0.0,0.00,0,0,0,0,8,487,0,487
1,10100,San Francisco County,1,1001,U,94133,44,38.8,1.33,30,30,0,0,8,487,0,487
2,10100,San Francisco County,1,1002,U,94133,0,0.0,0.00,0,0,0,0,8,487,0,487
3,10100,San Francisco County,1,1003,U,94133,0,0.0,0.00,0,0,0,0,8,487,0,487
4,10100,San Francisco County,1,1004,U,94133,1,36.5,0.00,0,0,0,0,8,487,0,487


In [19]:
gini = dfs[-3][dfs[-3].columns[[6,10,-3,-1]]][dfs[-3].COUNTY == 'San Francisco County'] \
            .rename(columns={'AD4BE001':'gini_a', 'AD4BM001':'gini_b'})

    
#gini = gini[gini.COUNTY == 'San Francisco County']

for c in gini.columns:
    if (c!='COUNTY'):
        gini[c] = pd.to_numeric(gini[c])

gini.head()

#[print(i, c) for i,c in enumerate(dfs[-3].columns)]

,COUNTY,TRACTA,gini_a,gini_b
9846,San Francisco County,10100,0.5391,0.0686
9847,San Francisco County,10200,0.5417,0.0510
9848,San Francisco County,10300,0.5614,0.1198
9849,San Francisco County,10400,0.4978,0.0590
9850,San Francisco County,10500,0.5370,0.0495


In [29]:
ahs_df = ahs_df.merge(gini[gini.columns[[1,2,3]]], left_on='tract',
                      right_on='TRACTA',how='left') \
                    .drop(['TRACTA'],axis=1)

ahs_df.head()

,tract,county,blkgroup,census_block,rur_urb,zip,population,median_age,avg_household_size,num_housing_units,occupied,vacant,seasonal,num_under_18,total_hholds,hholds_pubassist,hholds_no_pubassist,gini_a,gini_b
0,10100,San Francisco County,1,1000,R,94133,0,0.0,0.00,0,0,0,0,8,487,0,487,0.5391,0.0686
1,10100,San Francisco County,1,1001,U,94133,44,38.8,1.33,30,30,0,0,8,487,0,487,0.5391,0.0686
2,10100,San Francisco County,1,1002,U,94133,0,0.0,0.00,0,0,0,0,8,487,0,487,0.5391,0.0686
3,10100,San Francisco County,1,1003,U,94133,0,0.0,0.00,0,0,0,0,8,487,0,487,0.5391,0.0686
4,10100,San Francisco County,1,1004,U,94133,1,36.5,0.00,0,0,0,0,8,487,0,487,0.5391,0.0686


In [31]:
get_column_report(ahs_df)

,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
0,tract,int64,197,10100,0,0.00
1,county,object,1,San Francisco County,0,0.00
3,census_block,int64,404,1000,0,0.00
4,rur_urb,object,2,R,0,0.00
5,zip,int64,28,94133,0,0.00
2,blkgroup,int64,8,1,6,0.08
17,gini_a,float64,191,0.5391,10,0.14
18,gini_b,float64,159,0.0686,10,0.14
14,total_hholds,int64,420,487,21,0.28
16,hholds_no_pubassist,int64,419,487,21,0.28


In [37]:
### deal with missing values in AHS dataframe for census blocks included in master df ##### 

# first lets get some info about the census tracts that contain missing values - 
# each tract has a bunch of blocks within it, and some of those blocks might be missing data
# so the idea is to calculate the median values across the tract, and note the percentage of the tract's 
# blocks which have null values; then if < 50% of the blocks have empty cells we just impute those with
# the tract medians, and otherwise just discard everything in that tract
def tract_report(tract_list):
    tract_rep = []
    for t in tract_list:
        name = str(t)
        # first, grab the rows of the census data with the track
        minidf = ahs_df[ahs_df.tract==t]
        num_nan = len(minidf[minidf.population==0])
        pct_nan = 100*num_nan/float(len(minidf))
        medianpop = minidf.population[minidf.population!=0].median()
        medianage = minidf.median_age[minidf.median_age!=0].median()
        median_numhouses = minidf.num_housing_units[minidf.population!=0].median()
        avg_household_size = minidf.avg_household_size[minidf.population!=0].mean()
        med_occupied = minidf.occupied[minidf.population!=0].median()
        med_vacant = minidf.vacant[minidf.population!=0].median()
        med_seasonal = minidf.seasonal[minidf.population!=0].median()
        
        avg_gini = minidf['gini_a'][minidf.gini_a!=0].mean()
        med_children = minidf.num_under_18[minidf.num_under_18!=0].median()
        
#         print('-'*75, '\ntract %s' % name, '\nnumber of nulls: %i' % num_nan, 
#                 '\n total number of blocks in the tract: %i' % len(minidf),
#                 '\n%2.2f percent of blocks have missing info' % pct_nan,
#                 '\nmedian tract population: %.0f' % medianpop,
#                 '\nmedian age: %.0f' % medianage,
#                 '\nmedian number of houses: %.0f' % median_numhouses,
#                 '\navg household size: %.2f ' % avg_household_size,
#                 '\nmedian number of occupied houses: %.0f' % med_occupied,
#                 '\nmedian number of vacant houses: %.0f' % med_vacant,
#                 '\nmedian number of seasonal buildings: %.0f' % med_seasonal
#                 )
        # maybe should create a df for these ??
        tract_rep.append([t, pct_nan, medianpop, medianage, median_numhouses,
                            avg_household_size, med_occupied, med_vacant, med_seasonal, 
                            avg_gini, med_children])
    cols=['tract','percent_nan', 'median_pop', 'median_age', 'median_numhouses',
            'avg_household_size', 'med_occupied', 'med_vacant', 'med_seasonal', 
            'avg_gini', 'med_children']
    tract_rep = pd.DataFrame(tract_rep, columns=cols)
    return tract_rep

        
# the idea is that this will return a dataframe with the tract characteristics for all of the 
# tracts with missing data. then we iterate through the rows of ahs_df with missing population etc. data
# and if the tract has <75% empty values, we fill in the vals with the tract averages
# otherwise, if over 75% is missing, we discard all the data from that tract
tractreport = tract_report(ahs_df.tract.unique())


tractreport.head(10)

,tract,percent_nan,median_pop,median_age,median_numhouses,avg_household_size,med_occupied,med_vacant,med_seasonal,avg_gini,med_children
0,10100,37.500000,52.5,37.55,32.5,1.250333,32.0,1.0,0.0,0.5391,8.0
1,10200,39.534884,167.0,38.95,113.0,1.533846,88.0,15.0,3.5,0.5417,84.0
2,10300,7.142857,139.5,38.40,85.0,1.897308,75.0,11.0,2.0,0.5614,119.5
3,10400,11.764706,135.5,40.45,88.5,1.860000,73.5,12.5,3.0,0.4978,99.0
4,10500,54.000000,3.0,48.80,0.0,0.777826,0.0,0.0,0.0,0.5370,91.0
5,10600,6.896552,143.0,42.50,74.0,1.910000,67.0,10.0,1.0,0.5699,131.0
6,10700,12.000000,206.5,52.20,118.5,2.136818,109.5,9.5,1.0,0.6054,181.0
7,10800,8.333333,178.0,37.75,101.5,1.949091,85.0,12.0,3.0,0.4552,137.5
8,10900,8.333333,245.0,35.20,158.0,1.744545,144.0,11.5,2.0,0.4915,65.0
9,11000,18.518519,172.0,33.75,114.5,1.994545,102.5,8.0,0.0,0.4660,136.0


In [44]:

# so what I'm thinking is that we fill the missing population, num_housing_units, median_age, avg_household, tot_rooms etc.
# with the mean/median for that tract
# we'll want to do that with the ahs_df before merging into masterdf

#  make a dictionary of column_name:tract_characteristic and then iterate through and assign
# tractreport[tractreport.tract==tract].
               
cols_tofill = {'population':'median_pop', 'median_age':'median_age',
                'num_housing_units':'median_numhouses','avg_household_size':'avg_household_size', 
                'occupied':'med_occupied', 'vacant':'med_vacant', 'seasonal':'med_seasonal',
                'gini_a':'avg_gini', 'num_under_18':'med_children'}



# creating a new census dataframe to avoid confusion
# reset index so that can match up dataframes
ahs_filled = ahs_df.reset_index().copy()

before = int(len(ahs_filled))

# now we want to fill the missing values (the block,tract's that are in the overlaps df)


for row in ahs_df.reset_index().iterrows(): #.merge(overlaps, on=['census_block', 'tract'], how='inner')
    tract = row[1]['tract']
    if (tract in list(tractreport.tract)):
        report = tractreport[tractreport.tract==tract]
        # if avg_household_size is zero, most of the other ones will be too.
        if (row[1]['avg_household_size']==0):
            # we only want to impute vals if we have over 75% of the tract info
            if (report['percent_nan'].item() < 75.0):
                for c in cols_tofill:
                    # then check if the value is missing for each of the columns
                    if (row[1][c]==0):
                        # and if so, replace with the tract mean 
                        ahs_filled.loc[row[0], c] = report[cols_tofill[c]].item()
            # if we don't have much info for that tract & the value is missing, just drop it
            else:
                ahs_filled = ahs_filled.drop(row[0])


print(before - len(ahs_filled), 'rows dropped')
        

447 rows dropped


In [54]:
get_column_report(ahs_filled)

,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
9,avg_household_size,float64,564,1.25033,0,0.00
18,gini_a,float64,190,0.5391,0,0.00
8,median_age,float64,483,37.55,0,0.00
7,population,float64,578,52.5,0,0.00
6,zip,int64,28,94133,0,0.00
19,gini_b,float64,158,0.0686,0,0.00
4,census_block,int64,317,1000,0,0.00
3,blkgroup,int64,7,1,0,0.00
2,county,object,1,San Francisco County,0,0.00
1,tract,int64,195,10100,0,0.00


In [69]:
masterdf = pd.read_csv('data/molly_master_11152017.csv', low_memory=False)


print(masterdf.columns)



Index(['EAS', 'Incident_Dummy', 'Yr_Property_Built', 'Num_Stories',
       'Num_Units', 'Perc_Ownership', 'Land_Value', 'Property_Area',
       'Assessed_Improvement_Val', 'Tot_Rooms', 'landval_psqft',
       'count_potential_fire_control', 'count_all_complaints', 'tract',
       'census_block', 'latlong', 'inde', 'county', 'blkgroup', 'rur_urb',
       'zip', 'population', 'median_age', 'avg_household_size',
       'num_housing_units', 'occupied', 'vacant', 'seasonal', 'Neighborhood'],
      dtype='object')


In [80]:

# bc the master df has mostly float values
# for cens in ['tract','census_block']:
#     ahs_filled[cens] = ahs_filled[cens].apply(lambda x: float(x))


master_plus_census = pd.merge(masterdf, ahs_filled[ahs_filled.columns[[1,4,14,15,16,17,18,19]]], 
                              on=['census_block', 'tract'], how='inner')

master_plus_census.head()

,EAS,Incident_Dummy,Yr_Property_Built,Num_Stories,Num_Units,Perc_Ownership,Land_Value,Property_Area,Assessed_Improvement_Val,Tot_Rooms,...,occupied,vacant,seasonal,Neighborhood,num_under_18,total_hholds,hholds_pubassist,hholds_no_pubassist,gini_a,gini_b
0,451005.0,1.0,1989.0,3.0,2.777778,1.000000,438434.333333,4135.0,262181.666667,11.000000,...,60.0,1.0,0.0,SUNSET/PARKSIDE,181.0,432,17,415,0.4077,0.038
1,325727.0,1.0,1931.0,1.0,1.000000,0.740736,28593.888889,1320.0,118937.666667,5.000000,...,60.0,1.0,0.0,MISSION,181.0,432,17,415,0.4077,0.038
2,274924.0,0.0,1988.0,3.0,2.777778,1.000000,819727.777778,10863.0,441390.333333,27.000000,...,60.0,1.0,0.0,MISSION,181.0,432,17,415,0.4077,0.038
3,275686.0,0.0,1988.0,3.0,2.777778,1.000000,819727.777778,10863.0,441390.333333,27.000000,...,60.0,1.0,0.0,MISSION,181.0,432,17,415,0.4077,0.038
4,325724.0,0.0,1931.0,1.0,1.000000,1.000000,695324.222222,2450.0,337784.777778,10.666667,...,60.0,1.0,0.0,MISSION,181.0,432,17,415,0.4077,0.038


In [81]:
#ahs_filled.to_csv('data/master_census_file.csv', columns=ahs_filled.columns)

master_plus_census.to_csv('data/molly_master_11152017.csv', 
                          columns=master_plus_census.columns)

In [82]:
get_column_report(master_plus_census)

,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
0,EAS,float64,182233,451005,0,0.00
29,num_under_18,float64,279,181,0,0.00
23,avg_household_size,float64,458,3.3,0,0.00
22,median_age,float64,407,32,0,0.00
21,population,float64,535,198,0,0.00
20,zip,int64,26,94122,0,0.00
19,rur_urb,object,1,U,0,0.00
18,blkgroup,int64,7,3,0,0.00
33,gini_a,float64,187,0.4077,0,0.00
16,inde,int64,4926,533331,0,0.00
